In [2]:
import os
import pandas as pd
import numpy as np
import hashlib
import time
import copy
from IPython.display import clear_output


In [3]:
clustering_script = [
'#!/bin/bash',

'#SBATCH -J mad_MAD_NUM_K_IN',
'#SBATCH -p NODE_NUM',
'#SBATCH -N 1',
'#SBATCH -c THREAD_NUM',
'#SBATCH -o OUTPUT_Consensus.%j.out',
'#SBATCH -e OUTPUT_Consensus.%j.err',

'python NMF_clustering.py MAD_NUM K_IN SAVE_LOC DF_NAME FILE_PRI FILTER_FILE'
]


In [4]:
Temp_loc = "/data/Storage_DAS02/jaeyoon/230720_Muscle_Project/Code/241106_RNAseq_analysis_final/241106_NMF_results/"

ks = [i for i in range(2,11)]
# mad_idxs = [40,30,20,10]
mad_idxs = [40,50,60,70]

In [25]:
!scancel {77238..77239}

In [22]:
ks = [10]
# mad_idxs = [40,30,20,10]
mad_idxs = [60]

In [11]:
import re

In [23]:
# consensus clustering 돌리는 코드
# DF_NAME = '241206_data_integrated_disease_mean_centering_no_combat_gene_filtered.csv'
DF_NAME = '241206_data_integrated_normal_mean_centering_no_combat.csv'

# DF_NAME = '241206_data_integrated_normal_by_disease_mean_centering_no_combat_gene_filtered.csv'
# FILE_PRI = '241112_new_analysis_all'
# DF_NAME = '241112_integration_re_normal_combated_filtered.csv'
# FILE_PRI = '241209_analysis_mean_centering_disease_no_negative'
FILE_PRI = '241217_analysis_mean_centering_normal_no_negative'
# FILE_PRI = '241206_analysis_mean_centering_normal_by_disease'
# DF_NAME = '241112_integration_re_disease_combated_filtered.csv'
# FILE_PRI = '241112_new_analysis_disease_only'
# FILE_PRI = '240927_new_table_MAD10k2_table_stage2_C2_qnormed'
# FILTER_FILE = '240827_NMF_results/240927_MAD10k2_table_C2_qnormed.csv'
FILTER_FILE = "False"
node_past = ''
for mad_idx in mad_idxs:
    for k in ks:
        while True:
            A = !sinfo --Format=Partition,CPUsState
            A = A[1:12]
            B = !squeue
            pending_list = set(["node"+re.findall(r'node(\d+)',i)[0] for i in B if i.find('PD')>-1])
            node_state = {}
            for line in [i.split() for i in A]:
                # if line[0] not in ['node11','node8','node9']:
                if line[0] not in pending_list:
                    if int(line[1].split('/')[1])>0:
                        node_state[line[0]] = int(line[1].split('/')[1])
            node_name = ''
            if len(node_state)>0:
                if mad_idx >= 40:
                    # thread_num = 64
                    thread_num = 64
                else:
                    thread_num = 32
                    # thread_num = 24
                    
                for i,j in node_state.items():
                    if j >= thread_num:
                        if i != node_past:
                            node_name = i
                            node_past = i[:]
                            break
                if len(node_name) == 0:
                    for i,j in node_state.items():
                        if j >= thread_num//4*2:
                            if i != node_past:
                                node_name = i
                                thread_num = j
                                node_past = i[:]
                                break
            if len(node_name) != 0:
                break
            else:
                node_past = ''
                time.sleep(10)
                
        script = [i.replace('OUTPUT', os.path.join(Temp_loc,f'Consensus_{k}_{mad_idx}'))
                        .replace('SAVE_LOC', Temp_loc)
                        .replace('NODE_NUM',node_name)
                        .replace('THREAD_NUM',str(thread_num))
                        .replace('FILE_ID',f'Consensus_{k}_{mad_idx}')
                        .replace('K_IN', str(k))
                        .replace('DF_NAME', DF_NAME)
                        .replace('FILE_PRI', FILE_PRI)
                        .replace('MAD_NUM',str(mad_idx))
                        .replace('FILTER_FILE',FILTER_FILE) for i in clustering_script]
        sh_file_name = os.path.join(Temp_loc,f'Consensus_{k}_{mad_idx}.sh')
        with open(sh_file_name,'w') as f:
            f.write('\n'.join(script))
        run_info = !sbatch {sh_file_name}
        time.sleep(5)
        print(node_name,k,mad_idx, thread_num, run_info)

node8 10 60 64 ['Submitted batch job 90181']


In [5]:
!scancel {78442}

In [39]:
file_list = [i for i in file_list if (i.find('pkl')>-1) and (i.find('Consensus')>-1)]
for file_name in file_list:
    break
file_name[file_name.find('240510'):]

'240510_NMF_Combated_MAD30_k_2.pkl'

In [2]:
from matplotlib.figure import Figure


In [8]:
fig = Figure((6,6), dpi = 100)

AttributeError: 'Figure' object has no attribute 'plot'

6.0

In [40]:
file_loc = '/data/Storage_DAS02/jaeyoon/230720_Muscle_Project/Code/240409_microarray_preprocessing_yoo/240510_NMF_results/'
file_list = os.listdir(file_loc)
file_list = [i for i in file_list if (i.find('pkl')>-1) and (i.find('Consensus')>-1)]
for file_name in file_list:
    old_path = os.path.join(file_loc,file_name)
    new_path = os.path.join(file_loc,file_name[file_name.find('240510'):])
    os.rename(old_path, new_path)

In [29]:
!scancel {4419}

In [14]:
# node_name = node_names[n%len(node_names)]
# k = ks[n%len(ks)]
# mad_idx = mad_idxs[n%len(mad_idxs)]
# thread_num = 64
# node_name,k,mad_idx = ['node8',2,30]
n = 0
for mad_idx in mad_idxs:
    for k in ks:        
        node_name = node_names[n%len(node_names)]
        n+= 1
        script = [i.replace('OUTPUT', os.path.join(Temp_loc,f'Consensus_{k}_{mad_idx}'))
                        .replace('NODE_NUM',node_name)
                        .replace('THREAD_NUM',str(thread_num))
                        .replace('FILE_ID',f'Consensus_{k}_{mad_idx}')
                        .replace('K_IN', str(k))
                        .replace('MAD_NUM',str(mad_idx)) for i in clustering_script]
        sh_file_name = os.path.join(Temp_loc,f'Consensus_{k}_{mad_idx}.sh')
        with open(sh_file_name,'w') as f:
            f.write('\n'.join(script))
        run_info = !sbatch {sh_file_name}
        print(node_name,k,mad_idx, run_info)

node5 2 30 ['Submitted batch job 4392']
node6 3 30 ['Submitted batch job 4393']
node7 4 30 ['Submitted batch job 4394']
node10 5 30 ['Submitted batch job 4395']
node8 6 30 ['Submitted batch job 4396']
node9 7 30 ['Submitted batch job 4397']
node5 8 30 ['Submitted batch job 4398']
node6 9 30 ['Submitted batch job 4399']
node7 10 30 ['Submitted batch job 4400']
node10 2 20 ['Submitted batch job 4401']
node8 3 20 ['Submitted batch job 4402']
node9 4 20 ['Submitted batch job 4403']
node5 5 20 ['Submitted batch job 4404']
node6 6 20 ['Submitted batch job 4405']
node7 7 20 ['Submitted batch job 4406']
node10 8 20 ['Submitted batch job 4407']
node8 9 20 ['Submitted batch job 4408']
node9 10 20 ['Submitted batch job 4409']
node5 2 10 ['Submitted batch job 4410']
node6 3 10 ['Submitted batch job 4411']
node7 4 10 ['Submitted batch job 4412']
node10 5 10 ['Submitted batch job 4413']
node8 6 10 ['Submitted batch job 4414']
node9 7 10 ['Submitted batch job 4415']
node5 8 10 ['Submitted batch job 4

In [55]:
file_list = os.listdir(Temp_loc)
[i for i in file_list if i.find('err')>-1]
[i for i in file_list if i.find('pkl')>-1 and i.find('all')>-1]

['240418_consensus_clustering_all_MAD10_k_4.pkl',
 '240418_consensus_clustering_all_MAD10_k_2.pkl',
 '240418_consensus_clustering_all_MAD10_k_5.pkl',
 '240418_consensus_clustering_all_MAD10_k_3.pkl']

In [342]:
file_list = os.listdir(Temp_loc)
files = [i.split('_')[1:3]+[1]+[i] for i in file_list if (i.find('err')>-1)]
pd.DataFrame([i for i in files if (int(i[-1].split('.')[-2])>3690)]).pivot_table(values = 2, columns=1, index = 0, aggfunc='count')


1,10,20,30
0,,,
10,2,1,1
2,1,1,1
3,1,1,1
4,1,1,1
5,1,1,1
6,1,1,1
7,1,1,1
8,2,1,1
9,2,1,1


In [343]:
file_list = os.listdir(Temp_loc)
pd.DataFrame([i.replace('.pkl','').split('_')[-3:]+[1] for i in file_list if i.find('pkl')>-1 and i.find('all')>-1]).pivot_table(values = 3, columns=2, index = 0, aggfunc='count')


2,10,2,3,4,5,6,7,8,9
0,,,,,,,,,
MAD10,5,5,5,5,5,5,5,5,5
MAD20,5,5,5,5,5,5,5,5,5
MAD30,5,5,5,5,5,5,5,5,5
